### Project 3 IMDB Part 2
- Paula Pipkin

In [1]:
import pandas as pd
import os, time,json
import matplotlib.pyplot as plt
import seaborn as sns

import tmdbsimple as tmdb 
from tqdm.notebook import tqdm_notebook



In [2]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_results_combined.csv.gz']

In [3]:
def get_movies_cert(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
           info['certification'] = c['certification']
    return info

In [4]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


In [5]:
with open('/Users/paula/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
tmdb.API_KEY =  login['api-key']

In [6]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('data/title_basics.csv.gz')
basics.head()



,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,126,Drama


In [7]:
YEARS_TO_GET = [2000,2001]
errors = [ ]


In [8]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
        #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movies_cert(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

print(f"- Total errors: {len(errors)}")






    











YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/205 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/241 [00:00<?, ?it/s]

- Total errors: 446


Concatenate dfs and organize data

In [9]:
df2001 = pd.read_csv('data/final_tmdb_data_2001.csv.gz')
df2001 = df2001[['imdb_id','budget', 'revenue', 'certification']]
df2001.head()

,imdb_id,budget,revenue,certification
0,0,NaN,NaN,NaN
1,tt0035423,48000000.0,76019048.0,PG-13
2,tt0114447,0.0,0.0,NaN
3,tt0118589,22000000.0,5271666.0,PG-13
4,tt0118652,1000000.0,0.0,R


In [10]:
df2000 = pd.read_csv('data/final_tmdb_data_2000.csv.gz')
df2000 = df2000[['imdb_id','budget', 'revenue', 'certification']]
df2000.head()

,imdb_id,budget,revenue,certification
0,0,NaN,NaN,NaN
1,tt0113026,10000000.0,0.0,NaN
2,tt0113092,0.0,0.0,NaN
3,tt0116391,0.0,0.0,NaN
4,tt0118694,150000.0,12854953.0,PG


In [11]:
final_df = pd.concat([df2000, df2001], axis=0)
final_df

,imdb_id,budget,revenue,certification
0,0,NaN,NaN,NaN
1,tt0113026,10000000.0,0.0,NaN
2,tt0113092,0.0,0.0,NaN
3,tt0116391,0.0,0.0,NaN
4,tt0118694,150000.0,12854953.0,PG
...,...,...,...,...
1285,tt7797670,0.0,0.0,NaN
1286,tt7797790,0.0,0.0,NaN
1287,tt8665056,0.0,0.0,NaN
1288,tt8795764,0.0,0.0,NR


In [16]:
final_df.to_csv('Data/tmdb_results_combined.csv.gz', compression="gzip", index=False)

- How many movies had at least some valid financial information (values > 0 for budget OR revenue)?

In [12]:
with_info = final_df.loc[(final_df['budget'] >0) | (final_df['revenue'] >0)]

len(with_info) #total of movies with one of the financial info

623

In [13]:
with_info.head()

,imdb_id,budget,revenue,certification
1,tt0113026,10000000.0,0.0,NaN
4,tt0118694,150000.0,12854953.0,PG
6,tt0119273,15000000.0,0.0,R
10,tt0120263,0.0,80334.0,NaN
11,tt0120467,120000.0,14904.0,R


In [17]:
with_info.to_csv('Data/tmdb_results_combined_clean.csv.gz', compression="gzip", index=False)

- I have created all the EDA's after removing lines missing information for revenue AND budget, in a different notebook